In [1]:
# import pre-processed data
import pandas as pd
import numpy as np

pre_processed_data_path = '../20_intermediate_files/'

In [3]:
# load original image data
image_data_path = '../00_source_data/shopee-product-matching/train_images/'
train_csv = pd.read_csv('../00_source_data/shopee-product-matching/train.csv')

In [4]:
# start by using ResNet50 pretrained on ImageNet
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K

